<a href="https://colab.research.google.com/github/nicolenair/capstone/blob/master/anno_auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install stanfordnlp

     |████████████████████████████████| 163kB 2.8MB/s 


In [0]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import os
import re
import stanfordnlp

In [0]:
books = ["gables", "avonlea", "island", "dreams"]
urls = ["https://www.gutenberg.org/files/45/45-0.txt", "https://www.gutenberg.org/files/47/47-0.txt", "https://www.gutenberg.org/files/51/51-0.txt", "http://www.gutenberg.org/cache/epub/544/pg544.txt"]

for i in range(len(books)):
    page = requests.get(urls[i])
    file = open("{}.txt".format(books[i]) , "w+")
    file.write(page.text)
    file.close()

file = open("gables.txt", "r")
gables = file.read()
file = open("avonlea.txt", "r")
avonlea = file.read()
file = open("island.txt", "r")
island = file.read()
file = open("dreams.txt", "r")
dreams = file.read()

In [0]:
#returns list of paragraphs in a book that can be innputted into train_classifier
def paragraph_list(book):
    current_total = 0
    chapter_lengths = []
    full_para_list = []
    lengths = [38, 0, 40, 0]
    if book==gables:
        n_chap = lengths[0]
    elif book == dreams:
        n_chap = lengths[2]
    for i in range(1, n_chap):
        if book == dreams:
            index_c1 = book[re.search("CHAPTER {}".format(i), book).span()[0] + 11: re.search("CHAPTER {}".format(i + 1), book).span()[0]]
        if book == gables:
            nums = ["I", "II","III","IV","V", "VI", "VII", "VIII", "IX", "X", "XI", "XII", "XIII","XIV", "XV", "XVI", "XVII", "XVIII", "XIX", "XX", "XXI", "XXII", "XXIII", "XXIV", "XXV", "XXVI","XXVII" ,"XXVIII","XXIX", "XXX", "XXXI", "XXXII", "XXXIII","XXXIV", "XXXV" ,"XXXVI", "XXXVII", "XXXVIII"]
            first1 = re.search("CHAPTER {}\W".format(nums[i-1]), gables).span()[0]
            first2 = re.search("CHAPTER {}\W".format(nums[i]), gables).span()[0]
            index_c1 = gables[re.search("CHAPTER {}\W".format(nums[i-1]), gables[first1 + 1:]).span()[0] + first1: re.search("CHAPTER {}\W".format(nums[i]), gables[first2 + 1:]).span()[0] + first2]
        index_c1 = re.sub("(\n)(\w)", " \g<2>", index_c1)
        
        chapter_lengths.append(len(index_c1) - current_total)
        split_para_c1 = [s.strip() for s in index_c1.splitlines() if len(s)>10]
        full_para_list.append(split_para_c1)
        current_total += len(index_c1)  
        
    return full_para_list

full_para_list = paragraph_list(dreams)

#data set, be very careful with cell!!!



def remove_names(full_para_list, target):
    for o, i in enumerate(full_para_list):
        for t, e in enumerate(i):
            for k in name_mapping.keys():
                if target[o][t]==name_mapping[k]:
                    full_para_list[o][t] = re.sub(k, "", full_para_list[o][t])
                    full_para_list[o][t] = re.sub(k.lower(), "", full_para_list[o][t])
    return full_para_list
#have a more comprehensive way names could appear
#also try to do the said thing, rather than just removing all names

full_para_list_dreams = paragraph_list(dreams)
full_para_list_gables = paragraph_list(gables)

In [0]:
stanfordnlp.download('en')   # This downloads the English models for the neural pipeline
nlp = stanfordnlp.Pipeline() # This sets up a default neural pipeline in English

('You', '3', 'nsubj')
('may', '3', 'aux')
('copy', '0', 'root')
('it', '3', 'obj')
(',', '6', 'punct')
('give', '3', 'conj')
('it', '6', 'obj')
('away', '6', 'advmod')
('or', '10', 'cc')
('re-use', '6', 'conj')
('it', '10', 'obj')
('under', '14', 'case')
('the', '14', 'det')
('terms', '10', 'obl')
('of', '19', 'case')
('the', '19', 'det')
('Project', '19', 'compound')
('Gutenberg', '19', 'compound')
('License', '14', 'nmod')
('included', '19', 'acl')
('with', '23', 'case')
('this', '23', 'det')
('eBook', '20', 'obl')
('or', '25', 'cc')
('online', '23', 'conj')
('at', '27', 'case')
('www.gutenberg.org', '23', 'nmod')
('.', '3', 'punct')


In [0]:
#agent

def return_nsubj(book, character="Anne"):
  sentences_with_agent_Anne = []
  sentences_with_agent = []
  if book=="gables":
    doc = nlp(gables)
  elif book=="dreams":
    doc = nlp(dreams)
  for i in list(doc.sentences):
    for e in i.dependencies:
  #       print(e[1])
      if e[1]=="nsubj" or e[1]=="agent" :
        if e[2].text==character:
          sentences_with_agent_Anne.append(" ".join([i.dependencies[a][2].text for a in range(len(i.dependencies)) if i.dependencies[a][2].text!=character]))
        else:
          sentences_with_agent.append(" ".join([i.dependencies[a][2].text for a in range(len(i.dependencies)) if i.dependencies[a][2].text!=character]))
  target = []
  target.extend([[1] for i in range(len(sentences_with_agent_Anne))])
  target.extend([[0] for i in range(len(sentences_with_agent))])
  sentences = []
  sentences.extend([[i] for i in sentences_with_agent_Anne])
  sentences.extend([[i] for i in sentences_with_agent])
  return sentences, target

In [0]:
# sentences_with_nsubj[0]

In [0]:
#patient

In [0]:
#attributes

In [98]:
# doc.sentences[0].dependencies[5][2].text

'EBook'